In [ ]:
!pip install mplfinance
#to create candle-stick charts

     |████████████████████████████████| 71kB 3.7MB/s 


In [ ]:
#if Rutvik:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/Comprehensive Project/'

Mounted at /content/gdrive
/content/gdrive/MyDrive/Comprehensive Project


In [ ]:
# if Parth:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/.shortcut-targets-by-id/1YqdFa0VgzGfO_qydt2yan1_-O-mFSzJ4/Comprehensive Project'

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1YqdFa0VgzGfO_qydt2yan1_-O-mFSzJ4/Comprehensive Project


In [ ]:
"""
!rm data1/train/buy/*
!rm data1/train/sell/*
!rm data1/train/no_change/*
"""

'\n!rm data1/train/buy/*\n!rm data1/train/sell/*\n!rm data1/train/no_change/*\n'

In [ ]:
from pandas_datareader import data
from numpy import genfromtxt
import matplotlib.pyplot as plt
import mplfinance.original_flavor as mpl_finance
import numpy as np
import uuid

In [ ]:
modes_list = ['next__day', 'next_3_days', 'next_5_days']
mode = modes_list[0]

In [ ]:
#to create folders in "test" folder based on ticker_list

import os
ticker_list = ["SBIN.NS", "BANKBARODA.NS", "AXISBANK.NS", "FEDERALBNK.NS", "ICICIBANK.NS", "INDUSINDBK.NS", "PNB.NS", "HDFCBANK.NS", "KOTAKBANK.NS"]
for t in ticker_list:
  os.mkdir('data2/' + mode + '/test/' + t)
  os.mkdir('data2/' + mode + '/test/' + t + '/buy')
  os.mkdir('data2/' + mode + '/test/' + t + '/sell')
  os.mkdir('data2/' + mode + '/test/' + t + '/no_change')

In [ ]:
ticker_list = ["SBIN.NS", "BANKBARODA.NS", "AXISBANK.NS", "FEDERALBNK.NS", "ICICIBANK.NS", "INDUSINDBK.NS", "PNB.NS", "HDFCBANK.NS", "KOTAKBANK.NS"]
print("CREATING TEST DATA :")
for t in ticker_list:
  print("   GET DATA: TICKER = ", t)
  df = data.DataReader(t, start='2010-1-1', end='2021-04-21', data_source='yahoo')

  buy_dir  = 'data2/' + mode + '/test/' + t + '/buy/'
  sell_dir = 'data2/' + mode + '/test/' + t + '/sell/'
  no_change_dir = 'data2/' + mode + '/test/' + t + '/no_change/'
  change_thres = 0.025

  def convolve_sma(array, period): #find moving average
      return np.convolve(array, np.ones((period,))/period, mode='valid')

  def graphwerk(start, finish):
      open, high, low, close, volume, date = [], [], [], [], [], []
      for _ in range(finish-start):
          open.append(df['Open'][start])
          high.append(df['High'][start])
          low.append(df['Low'][start])
          close.append(df['Close'][start])
          #volume.append(df['Volume'][start])
          date.append(df.index[start])
          start = start + 1

      if mode == modes_list[0]:
        close_next = float(df['Close'][finish])

      elif mode == modes_list[1]:
        next_3_avg = []
        for i in range(3):
          next_3_avg.append(float(df['Close'][finish + i]))
        next_3_avg = np.average(next_3_avg)
        close_next = next_3_avg

      elif mode == modes_list[2]:
        next_5_avg = []
        for i in range(5):
          next_5_avg.append(float(df['Close'][finish + i]))
        next_5_avg = np.average(next_5_avg)
        close_next = next_5_avg

      sma = convolve_sma(close, 5)
      smb = list(sma)
      diff = sma[-1] - sma[-2]

      for x in range(len(close)-len(smb)):
          smb.append(smb[-1]+diff)

      fig = plt.figure(num=1, figsize=(3, 3), dpi=50, facecolor='w', edgecolor='k')
      dx = fig.add_subplot(111)
      #mpl_finance.volume_overlay(ax, open, close, volume, width=0.4, colorup='b', colordown='b', alpha=1)
      mpl_finance.candlestick2_ochl(dx,open, close, high, low, width=1.5, colorup='g', colordown='r', alpha=0.5)

      plt.autoscale()
      plt.plot(smb, color="blue", linewidth=10, alpha=0.5)
      plt.axis('off')
      comp_ratio = close_next / close[-1]
      #print(comp_ratio)

      if close[-1] > close_next and abs((close[-1]-close_next)/close[-1]) > change_thres :
              #print('close value is bigger')
              #print('last value: ' + str(close[-1]))
              #print('next value: ' + str(close_next))
              #print('sell')
              plt.savefig(sell_dir + "diff=" + str(round(100*abs((close[-1]-close_next)/close[-1]),3)) + "___" + str(uuid.uuid4()) +'.jpg', bbox_inches='tight')

      elif abs((close[-1]-close_next)/close[-1]) > change_thres:
              #print('close value is smaller')
              #print('last value: '+ str(close[-1]))
              #print('next value: ' + str(close_next))
              #print('buy')
              plt.savefig(buy_dir + "diff=" + str(round(100*abs((close[-1]-close_next)/close[-1]),3)) + "___" + str(uuid.uuid4()) +'.jpg', bbox_inches='tight')
      else:
              #print('close value change is less')
              #print('last value: '+ str(close[-1]))
              #print('next value: ' + str(close_next))
              #print('no_change')
              plt.savefig(no_change_dir + "diff=" + str(round(100*abs((close[-1]-close_next)/close[-1]),3)) + "___" + str(uuid.uuid4())+'.jpg', bbox_inches='tight')

      #plt.show()
      open.clear()
      close.clear()
      volume.clear()
      high.clear()
      low.clear()
      plt.cla()
      plt.clf()
  iter_count = int(len(df)/4)
  iter = 0
  for x in range(len(df) - 4):
    if iter + 28 >= len(df):
      break

    graphwerk(iter, iter + 8)
    iter = iter + 2

CREATING TEST DATA :
   GET DATA: TICKER =  SBIN.NS
   GET DATA: TICKER =  BANKBARODA.NS
   GET DATA: TICKER =  AXISBANK.NS
   GET DATA: TICKER =  FEDERALBNK.NS
   GET DATA: TICKER =  ICICIBANK.NS
   GET DATA: TICKER =  INDUSINDBK.NS
   GET DATA: TICKER =  PNB.NS
   GET DATA: TICKER =  HDFCBANK.NS
   GET DATA: TICKER =  KOTAKBANK.NS


<Figure size 150x150 with 0 Axes>

In [ ]:
#merge all folders from "test" and create common "buy", "sell" and "no_change" folders
from distutils.dir_util import copy_tree
import os
print("CREATING TRAIN DATA :")
for t in ticker_list:
  for sclass in ['buy', 'sell', 'no_change']:
    copy_tree(str(os.getcwd()) + '/data2/' + mode + '/test/' + t + '/' + sclass + '/', str(os.getcwd()) + '/data2/' + mode + '/train/' + sclass + '/')

CREATING TRAIN DATA :


# Training

In [ ]:
import os
import sys
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import optimizers, initializers
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
img_width, img_height = 150, 150
datagen_train_validation = ImageDataGenerator(
                                   rescale=1./255,
                                   horizontal_flip = False,
                                   validation_split = 0.2)  
                                  
train_generator = datagen_train_validation.flow_from_directory(
                                  'data2/' + mode + '/train/',
                                  target_size = (img_width, img_height),
                                  seed = 5,
                                  class_mode = 'categorical',
                                  subset='training')

validation_generator = datagen_train_validation.flow_from_directory(
                                  'data2/' + mode + '/train/',
                                  target_size = (img_width, img_height),
                                  seed = 5,
                                  class_mode = 'categorical',
                                  subset = 'validation')

Found 9930 images belonging to 3 classes.
Found 2481 images belonging to 3 classes.


In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5, 5), activation='relu', padding = 'same', input_shape = (150, 150 , 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy', tf.keras.metrics.TruePositives(name="TP"), tf.keras.metrics.TrueNegatives(name="TN"), tf.keras.metrics.FalsePositives(name="FP"), tf.keras.metrics.FalseNegatives(name="FN")])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 150, 150, 32)      2432      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 75, 75, 32)        128       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 37, 37, 64)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 37, 37, 64)        256       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 37, 37, 128)      

In [ ]:
model = keras.models.load_model(str(os.getcwd()) + '/models/' + mode)

In [ ]:
#target_dir = "models/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
#if not os.path.exists(target_dir):
#  os.mkdir(target_dir)
#model.save('models/model.h5')
#model.save_weights('models/weights.h5')

#checkpoint = ModelCheckpoint(target_dir, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

history = model.fit(
    x = train_generator,
    steps_per_epoch=16,
    epochs = 8,
    shuffle=False,
    validation_data=validation_generator
    #callbacks=callbacks_list
    )

#img_res=(150,150)  next_day__1  val_acc=85
#img_res=(200,200)  next_days_3  val_acc=76
#img_res=(150,150)  next_days_3  val_acc=76
#img_res=(150,150)  next_days_5  val_acc=69

Epoch 1/8
16/16 [==============================] - 138s 9s/step - loss: 2.7388 - accuracy: 0.7075 - TP: 145.4118 - TN: 524.0000 - FP: 48.2353 - FN: 140.7059 - val_loss: 0.6600 - val_accuracy: 0.8130 - val_TP: 2017.0000 - val_TN: 4498.0000 - val_FP: 464.0000 - val_FN: 464.0000
Epoch 2/8
16/16 [==============================] - 138s 9s/step - loss: 1.2930 - accuracy: 0.7627 - TP: 34.2353 - TN: 558.3529 - FP: 13.8824 - FN: 251.8824 - val_loss: 0.7324 - val_accuracy: 0.8130 - val_TP: 2017.0000 - val_TN: 4498.0000 - val_FP: 464.0000 - val_FN: 464.0000
Epoch 3/8
16/16 [==============================] - 136s 8s/step - loss: 1.0731 - accuracy: 0.7235 - TP: 1.8824 - TN: 552.8824 - FP: 19.3529 - FN: 284.2353 - val_loss: 0.9915 - val_accuracy: 0.8130 - val_TP: 5.0000 - val_TN: 4961.0000 - val_FP: 1.0000 - val_FN: 2476.0000
Epoch 4/8
16/16 [==============================] - 131s 8s/step - loss: 1.0544 - accuracy: 0.8227 - TP: 0.0000e+00 - TN: 572.2353 - FP: 0.0000e+00 - FN: 286.1176 - val_loss: 0.

In [ ]:
!ls models -l

In [ ]:
model.save(str(os.getcwd()) + '/models/' + mode + '__banks_26Apr_3')

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/Comprehensive Project/models/next__day__banks_26Apr_3/assets


# Testing on Individual Tickers

In [ ]:
modes_list = ['next__day', 'next_3_days', 'next_5_days']
mode = modes_list[0]
model = keras.models.load_model(str(os.getcwd()) + '/models/' + mode + '__bank_8')

In [ ]:
"""
import os
ticker_list = ["SBIN.NS", "BANKBARODA.NS", "AXISBANK.NS", "FEDERALBNK.NS", "ICICIBANK.NS", "INDUSINDBK.NS", "PNB.NS", "HDFCBANK.NS", "KOTAKBANK.NS"]
for t in ticker_list:
  os.mkdir('data1/' + mode + '/test/' + t)
  os.mkdir('data1/' + mode + '/test/' + t + '/buy')
  os.mkdir('data1/' + mode + '/test/' + t + '/sell')
  os.mkdir('data1/' + mode + '/test/' + t + '/no_change')
"""

In [ ]:
"""
import shutil
shutil.rmtree()
for t in ticker_list:
  shutil.rmtree('data/' + mode + '/test/' + t + '/buy/')
  shutil.rmtree('data/' + mode + '/test/' + t + '/sell/')
  shutil.rmtree('data/' + mode + '/test/' + t + '/no_change/')
"""

In [ ]:
ticker_list = ["SBIN.NS", "BANKBARODA.NS", "AXISBANK.NS", "FEDERALBNK.NS", "ICICIBANK.NS", "INDUSINDBK.NS", "PNB.NS", "HDFCBANK.NS", "KOTAKBANK.NS"]

In [ ]:
filenames_dict, predict_dict = {}, {}

for t in ticker_list:
  print("Testing for : ", t, "  mode : ", mode )
  img_width, img_height = 150, 150
  datagen_test = ImageDataGenerator(
                                  rescale=1./255,
                                  horizontal_flip = False,
                                  validation_split = False)  
                                    
  test_generator = datagen_test.flow_from_directory(
                                    'data2/' + mode + '/test/' + t,
                                    target_size = (img_width, img_height),
                                    seed = 5,
                                    shuffle = False,
                                    class_mode = 'categorical',
                                    subset=None)
  filenames = test_generator.filenames
  predict = model.evaluate(x=test_generator)
  filenames_dict[t] = filenames
  predict_dict[t] = predict

Testing for :  SBIN.NS   mode :  next__day
Found 1379 images belonging to 3 classes.
44/44 [==============================] - 260s 6s/step - loss: 0.8112 - accuracy: 0.8107 - TP: 757.0000 - TN: 2593.0000 - FP: 165.0000 - FN: 622.0000
Testing for :  BANKBARODA.NS   mode :  next__day
Found 1379 images belonging to 3 classes.
44/44 [==============================] - 278s 6s/step - loss: 0.9450 - accuracy: 0.7592 - TP: 755.0000 - TN: 2518.0000 - FP: 240.0000 - FN: 624.0000
Testing for :  AXISBANK.NS   mode :  next__day
Found 1379 images belonging to 3 classes.
44/44 [==============================] - 288s 7s/step - loss: 0.8662 - accuracy: 0.7839 - TP: 720.0000 - TN: 2562.0000 - FP: 196.0000 - FN: 659.0000
Testing for :  FEDERALBNK.NS   mode :  next__day
Found 1379 images belonging to 3 classes.
44/44 [==============================] - 283s 7s/step - loss: 0.8767 - accuracy: 0.7839 - TP: 739.0000 - TN: 2552.0000 - FP: 206.0000 - FN: 640.0000
Testing for :  ICICIBANK.NS   mode :  next__day


In [ ]:
"""
8 Days Window
TICKER           next_day

SBIN.NS       :   81.28
BANKBARODA.NS :   75.92
AXISBANK.NS   :   78.34
CENTRALBK.NS  :   78.12
HDFCBANK.NS   :   92.36
INDIANB.NS    :   72.91
KOTAKBANK.NS  :   86.78
UNIONBANK.NS  :   68.58
CANBK.NS      :   70.78
IOB.NS        :   78.78
"""

In [ ]:
"""
20 Days Window
TICKER           next_day

SBIN.NS       :   81.28
BANKBARODA.NS :   75.92
AXISBANK.NS   :   78.56  
CENTRALBK.NS  :   78.12
HDFCBANK.NS   :   92.44
INDIANB.NS    :   72.61
KOTAKBANK.NS  :   86.78  
UNIONBANK.NS  :   68.50
CANBK.NS      :   70.70
IOB.NS        :   78.78
"""

In [ ]:
"""
12 Days Window
TICKER        next_5_days   next_3_days   next_day

SBIN.NS       :   60.38       67.89         70.80
BANKBARODA.NS :   55.52       63.62         65.68
ADANIPOWER.NS :   50.66       58.32         60.64
RELIANCE.NS   :   69.37       78.94         78.50
HDFCBANK.NS   :   80.71       86.38         86.60
TCS.NS        :   74.23       81.96         81.96
INFY.NS       :   76.73       84.54         83.54
HDFC.NS       :   76.95       82.77         82.62
BHARTIARTL.NS :   66.86       73.49         74.04
WIPRO.NS      :   76.29       82.11         83.06
ULTRACEMCO.NS :   72.75       79.82         80.04
TITAN.NS      :   65.54       72.61         73.75


"""